In [1]:
import tensorflow as tf
import keras
import numpy as np

import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

Using TensorFlow backend.


In [2]:
tf.__version__

'2.0.0'

In [3]:
from tensorflow.keras import layers
from tensorflow.keras import layers
import datetime
logdir = "logs/scalars/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)


model = tf.keras.Sequential()
model.add(layers.Dense(400,input_shape=(9,), activation='relu'))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(2, activation = 'softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [5]:
class tune_param_classifier:
    def __init__(self, x, y, epoch,learning_rate, no_of_layers, 
                 batch_size,validation_data, metric = 'accuracy'):
        self.x_dimension = x.shape
        self.target_dimension = y.shape
        
        #self layers range (start, stop, step)
        self.layers_range = (100,1000,50)
        
    
        #no of possiblities
        self.possibilities = self.no_of_possibilities(len(self.layers_range))
        
        # evaluate network
        self.evaluate_network(x,y,epoch,no_of_layers,learning_rate,
                              batch_size,metric, validation_data)
                
        return
    
    
    def no_of_posibilities(self, no_of_layers):
        count = 1
        for i in range(1, no_of_layers+1):
            count *= i
        return count
    
    
    def create_network(self, neurons):
        model = tf.keras.Sequential()
        
        # iterate through the dict layers
        model.add(layers.Dense(neurons[0],input_shape=(self.x_dimension[1],), activation='relu'))
        
        # add hidden layers
        for i in range(neurons[1:]):
            model.add(layers.Dense(i, activation='relu'))
            
        # add output layers
        if self.target_dimension[1] == None:
            model.add(layers.Dense(1, activation = 'softmax'))
        else:
            model.add(self.target_dimension[1], activation = 'softmax')
            
        return model
    
    def metrics_precision(self,actual, prediction):
        prediction = tf.argmax(logits, 1)
        actual = tf.argmax(y,1)
    
        TP = tf.math.count_nonzero(prediction * actual)
        TN = tf.math.count_nonzero((prediction - 1) * (actual - 1))
        FP = tf.math.count_nonzero(prediction * (actual - 1))
        FN = tf.math.count_nonzero((prediction - 1) * actual)

        precision = TP/(TP+FP)
        return precision
    
    
    def metric_accuracy(self, actual, prediction):
        prediction = tf.argmax(logits, 1)
        actual = tf.argmax(y,1)
        
        TP = tf.math.count_nonzero(prediction * actual)
        TN = tf.math.count_nonzero((prediction - 1) * (actual - 1))
        FP = tf.math.count_nonzero(prediction * (actual - 1))
        FN = tf.math.count_nonzero((prediction - 1) * actual)

        accuracy = (TP+TN)/(TP+TN+FP+FN)
        return accuracy
    
    def metric_recall(self, actual, prediction):
        prediction = tf.argmax(logits, 1)
        actual = tf.argmax(y,1)
        
        TP = tf.math.count_nonzero(prediction * actual)
        TN = tf.math.count_nonzero((prediction - 1) * (actual - 1))
        FP = tf.math.count_nonzero(prediction * (actual - 1))
        FN = tf.math.count_nonzero((prediction - 1) * actual)

        Recall = TP/(TP+FN)
        return Recall
    
    
    def metric_f1_score(self, actual, prediction):
        prediction = tf.argmax(logits, 1)
        actual = tf.argmax(y,1)
        
        TP = tf.math.count_nonzero(prediction * actual)
        TN = tf.math.count_nonzero((prediction - 1) * (actual - 1))
        FP = tf.math.count_nonzero(prediction * (actual - 1))
        FN = tf.math.count_nonzero((prediction - 1) * actual)

        precision = TP/(TP+FP)
        Recall = TP/(TP+FN)
        F1_Score = 2*(Recall * precision) / (Recall + precision)
        return F1_Score
    
    
    
    
    def evaluate_network(self,x,y,epoch,no_of_layers,
                         learning_rate,batch_size,metric, 
                         validation_data):
        # ranges of neuron values
        data = [i for i in range(self.layers_range)]
        
        #optimizers
        optimize = {'rmsprop':tf.keras.optimizers.RMSprop(learning_rate),
                     'gradient descent':tf.keras.optimizers.SGD(learning_rate),
                     'adam':tf.keras.optimizers.Adam(learning_rate),
                     'adagrad':tf.keras.optimizers.AdaGrad(learning_rate),
                     'adadelta':tf.keras.optimizers.AdaDelta(learning_rate),
                     'nadem':tf.keras.optimizers.Nadem(learning_rate)}
        
        
        # losses
        losses = {'binary':tf.keras.losses.binary_crossentropy(0.01),
               'categorical':tf.keras.losses.categorical_crossentropy(logit = True)}
        
        

        dict_neurons = {}
        for i,pos in enumerate(range(self.possibilities)):
            current_neurons = np.random.choice(data, no_of_layers, replace = False)
            # take record of all neurons
            dict_neurons[str(i)] = current_neurons
            
            #create network for the neurons
            model = self.create_network(current_neurons)
            
            for m in optimize.keys():
                if (y.shape[1] in [None, 1]):
                    model.compile(optimizer=optimize[m],
                                  loss= losses['binary'],
                                  metrics=['accuracy'])
                    model.fit(x,y, epoch = epoch, batch_size= batch_size,
                             validation_data = validation_data, verbose = 0)
                else:
                    y.shape[1]>1:
                    model.compile(optimizer = optimize[m],
                                  loss= losses['categorical'],
                                  metrics=['accuracy'])
                    h = model.fit(x,y, epoch = epoch, batch_size= batch_size,
                             validation_data = validation_data,verbose =0)
               
                    
            
        return scores

In [5]:
def possibilities(dict_array):
    
    return